In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
nltk.download('stopwords')  # Download NLTK data if needed
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:

df = pd.read_csv("train.csv")
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [36]:
# Function to clean and preprocess text
def preprocess_text(text):
    text = text.lower()
    import string
    text = text.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text



# Sample text
text = "This is an example sentence, with some punctuation and common English stopwords."

# Apply preprocessing
cleaned_text = preprocess_text(text)

# Output the cleaned text
print("Original text:", text)
print("Cleaned text:", cleaned_text)


Original text: This is an example sentence, with some punctuation and common English stopwords.
Cleaned text: example sentence punctuation common english stopwords


In [37]:
# Apply preprocessing to the "comment_text" column
df["comment_text"] = df["comment_text"].apply(preprocess_text)


In [38]:
# Define negative labels
negative_labels = ["toxic", "severe_toxic", "obscene", "threat", "identity_hate", "insult"]
print(df.columns)


Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')


In [39]:
# Select rows where any of the negative categories have a value of 1
subset_size=10000;
df_negative = df[(df["toxic"] == 1) | (df["severe_toxic"] == 1) | (df["obscene"] == 1) | 
(df["insult"] == 1) | (df["threat"] == 1) | (df["identity_hate"] == 1)].sample(n=subset_size, random_state=42)

# Select rows where all negative categories have a value of 0
df_positive = df[(df["toxic"] == 0) & (df["severe_toxic"] == 0) & (df["obscene"] == 0) & 
(df["insult"] == 0) & (df["threat"] == 0) & (df["identity_hate"] == 0)].sample(n=subset_size, random_state=42)

# Print the status for the top 10 data points
print("Top 10 data points and their status:")
print("-------------------------------------")
for i, (index, row) in enumerate(df.head(10).iterrows()):
    if index in df_negative.index:
        status = "Negative"
    else:
        status = "Positive"
    print(f"Data point {i+1}: {status}")
    print(row)  # Optionally print the data point
    print("-------------------------------------")

Top 10 data points and their status:
-------------------------------------
Data point 1: Positive
id                                                0000997932d777bf
comment_text     explanation edits made username hardcore metal...
toxic                                                            0
severe_toxic                                                     0
obscene                                                          0
threat                                                           0
insult                                                           0
identity_hate                                                    0
Name: 0, dtype: object
-------------------------------------
Data point 2: Positive
id                                                000103f0d9cfb60f
comment_text     daww matches background colour im seemingly st...
toxic                                                            0
severe_toxic                                                     0
obscene       

In [40]:

7.# Create TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
features_negative = vectorizer.fit_transform(df_negative["comment_text"])
features_positive = vectorizer.transform(df_positive["comment_text"])

In [41]:
# Split data into training and testing sets for each class
X_train_negative, X_test_negative, y_train_negative, y_test_negative = train_test_split(
    features_negative, df_negative["comment_text"], test_size=0.2
)
X_train_positive, X_test_positive, y_train_positive, y_test_positive = train_test_split(
    features_positive, df_positive["comment_text"], test_size=0.2
)


In [42]:
# Split data into training and testing sets for each class
X_train_negative, X_test_negative, y_train_negative, y_test_negative = train_test_split(
    features_negative[:10000], df_negative["comment_text"][:10000], test_size=0.2
)
X_train_positive, X_test_positive, y_train_positive, y_test_positive = train_test_split(
    features_positive[:10000], df_positive["comment_text"][:10000], test_size=0.2
)


In [43]:

# Train Separate Naive Bayes Models (consider regularization if needed)
model_negative = MultinomialNB(alpha=0.1)  # Adjust alpha as needed
model_positive = MultinomialNB(alpha=0.1)  # Adjust alpha as needed
model_negative.fit(X_train_negative, y_train_negative)	
model_positive.fit(X_train_positive, y_train_positive)


MultinomialNB(alpha=0.1)

In [44]:
# Classify based on thresholds
import numpy as np
negative_threshold = 0.75  # Adjust based on evaluation
positive_threshold = 0.75  # Adjust based on evaluation
y_pred_final = np.where(y_pred_negative > negative_threshold, "negative",
np.where(y_pred_positive > positive_threshold, "positive", "unknown"))
import numpy as np


#example ko lagi
# Sample predicted probabilities for the negative and positive classes
  #y_pred_negative = np.array([0.8, 0.6, 0.3])  # Sample negative probabilities
  #y_pred_positive = np.array([0.2, 0.4, 0.7])  # Sample positive probabilities
#3ta comment ko negatice pred 0.8, 0.6 ra 0.3 cha where 0.8>0.5 so  it is negative
# Thresholds for classification
  #negative_threshold = 0.5
  #positive_threshold = 0.5

# Classify based on thresholds
  #y_pred_final = np.where(y_pred_negative > negative_threshold, "negative",
  #np.where(y_pred_positive > positive_threshold, "positive", "unknown"))

# Print the final classifications
  #for i, pred in enumerate(y_pred_final):
  #print(f"Test Sample {i+1}: {pred}")


In [45]:
y_pred_final = np.where(y_pred_negative > negative_threshold, "negative",
np.where(y_pred_positive > positive_threshold, "positive", "unknown"))


In [46]:
# Evaluate performance for both classes
precision_negative = precision_score(y_test_negative, y_pred_final, average='macro', zero_division=0)
recall_negative = recall_score(y_test_negative, y_pred_final, average='macro', zero_division=0)
f1_negative = f1_score(y_test_negative, y_pred_final, average='macro', zero_division=0)

precision_positive = precision_score(y_test_positive, y_pred_final, average='macro', zero_division=0)
recall_positive = recall_score(y_test_positive, y_pred_final, average='macro', zero_division=0)
f1_positive = f1_score(y_test_positive, y_pred_final, average='macro', zero_division=0)

ValueError: Found input variables with inconsistent numbers of samples: [2000, 3]